In [27]:
import pandas as pd
import os
import requests
from dotenv import load_dotenv
import time

In [26]:
# %pip install python-dotenv

In [7]:
load_dotenv()
RIOT_API_KEY = os.getenv("RIOT_API_KEY")

In [31]:
apex_tiers = ['challengerleagues', 'grandmasterleagues', 'masterleagues']
#divisions = ['DIAMOND', "EMERALD"]
divisions = ['DIAMOND']
tiers = ["I", "II", "III", "IV"]
#tiers = ["I"]

In [ ]:
#im just looking at the docs

In [9]:
#variable to input API key in the header of request instead of query
headers = {
    "X-Riot-Token" : RIOT_API_KEY
}

In [32]:
def get_summoner_ids_from_api(api_url, req_type):
    summoner_ids = []
    api_url = api_url
    try:
        resp = requests.get(api_url, headers=headers )  #Insert API link here
        resp.raise_for_status()
    except requests.HTTPError:
        print("Couldn't complete request")   
    response = resp.json()
    if req_type == "apex":
        entries = response['entries']
        for entry in entries:
            summoner_ids.append(entry['summonerId'])
    if req_type == "regular":
        for entry in response:
            summoner_ids.append(entry['summonerId'])
    return summoner_ids


def get_sums_for_apex_leagues():
    apex_summoner_ids = []
    for apex_league in apex_tiers:
        print(apex_league)
        api_url = f"https://na1.api.riotgames.com/lol/league/v4/{apex_league}/by-queue/RANKED_SOLO_5x5"
        summoner_ids = get_summoner_ids_from_api(api_url, "apex")
        apex_summoner_ids += summoner_ids
    return apex_summoner_ids

def get_sums_for_reg_divisions(start_time):
    inner_start_time = start_time
    total_loops = 3 #starts at 3 since there should be 3 apex tier calls before this
    reg_division_ids = []
    for division in divisions:
        print(division)
        for tier in tiers:
            print(tier)
            page = 1
            more_pages=True
            while more_pages:
                if total_loops == 100: #if we have made 100 requests
                    dif = time.time()-inner_start_time
                    if dif< 120: #if it has not been 120 seconds
                        print(f'sleeping for {dif}')
                        time.sleep(dif)
                        inner_start_time = time.time() #reset start time
                        total_loops = 0 #reset number of loops as well
                print(page)
                api_url = f'https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{division}/{tier}?page={page}'
                output_list = get_summoner_ids_from_api(api_url, "regular")
                if output_list:
                    page += 1
                    reg_division_ids += output_list
                else:
                    more_pages = False
                total_loops += 1 #keeping track of number of api calls for rate limiting purposes
    return reg_division_ids

def get_all_wanted_sums():
    all_sum_ids = []
    start = time.time()
    all_sum_ids += get_sums_for_apex_leagues()
    all_sum_ids += get_sums_for_reg_divisions(start)
    return all_sum_ids


#could later make this a dataframe the whole time, and add the rank information along with the summoner ID
# schema = {'summonerId': 'str', 'Division': 'str', 'Tier': 'str'}
# df = pd.DataFrame(columns=schema.keys()).astype(schema)

all_wanted_sum_ids = get_all_wanted_sums()
print(all_wanted_sum_ids)
print(len(all_wanted_sum_ids))
all_wanted_sum_ids = list(set(all_wanted_sum_ids))
print(len(all_wanted_sum_ids)) #see if it dropped any duplicates

#make df
sums_df = pd.DataFrame(all_wanted_sum_ids)
sums_df


masterleagues
grandmasterleagues
challengerleagues
DIAMOND
I
1
2
3
4
5
6
7
8
9
10
II
1
2
3
4
5
6
7
8
9
10
11
12
13
14
III
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
IV
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
EMERALD
I
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
II
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
III
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


SSLError: HTTPSConnectionPool(host='na1.api.riotgames.com', port=443): Max retries exceeded with url: /lol/league/v4/entries/RANKED_SOLO_5x5/EMERALD/III?page=21 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))

In [ ]:
sums_df.to_csv("all_wanted_summoner_ids.csv")

In [28]:
start = time.time()
time.sleep(5)
end = time.time()
dif = end-start
print(dif)

5.004995822906494


In [6]:

#.env variables; API key and my own IGN + tag
load_dotenv()
RIOT_API_KEY = os.getenv("RIOT_API_KEY")

#variable to input API key in the header of request instead of query
headers = {
    "X-Riot-Token" : RIOT_API_KEY
}


#placeholder function that might eventually call multiple functions whenif a purpose for this commit is established
def main():
    user_input = input("Please provide input") #What are we searching for
    search = get_function(query = user_input)  #Where we are searching it
    for user_input in search:
        print(f"*{user_input}")
#API call blueprint, will delete later
def get_function(query):
    try:
        resp = requests.get("", query)  #Insert API link here
        resp.raise_for_status()
    except requests.HTTPError:
        print("Couldn't complete request")  
    dataset = resp.json()
    return[iterable["parameter"] for iterable in dataset["key"]] #parameter = desired output (DOCUMENTATION) / key = where output is found (DOCUMENTATION)
     
#function to use RiotID (name + tag) to obtain unique player identifier (puuid) that is used a lot in many APIs
#tested and is functional
def get_puuid(game_name, tag_line):
    api_url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"
    try:
        resp = requests.get(api_url, headers=headers )  #Insert API link here
        resp.raise_for_status()
    except requests.HTTPError:
        print("Couldn't complete request")   
    player_info = resp.json()
    return player_info["puuid"]

#function to use puuid to get a list of the match ids of the last 20 games played
#parametes default to last 20 games but can go up to 100
#tested and is functional
def get_match_ids():
    puuid = get_puuid(my_game_name, my_tag_line)
    api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20"
    try:
        resp = requests.get(api_url, headers=headers)
        resp.raise_for_status()
    except requests.HTTPError:
        print("Couldn't complete request")
    match_list = resp.json()
    print(match_list[0])

get_match_ids()

def question_mark_ping():
    sample = get_match_ids()
    api_url = f""
    
    #return[iterable["parameter"] for iterable in player_info["key"]] 


#Create variable for match ids (list)
#Parameters= startTime, queue(int for id), type(string for queue type), count(into for number of match ids)

NameError: name 'my_game_name' is not defined